Cross Validation 5

size =  72684

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.0) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_train_5.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_72684/CV_df_val_5.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(58147, 23)
(14537, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Obstetrics & Gynaecology,com.andromo.dev706301.app837242,Education,4.4,36419,False,Obstetrics & Gynaecology application is mainly...,https://gist.githubusercontent.com/MedicoGuide...,0,0,...,0,0,0,0,0,0,152,obstetrics gynaecology application is mainly d...,150,33
1,FordPass,com.ford.fordpass,Lifestyle,4.3,3064680,False,FordPass has so many ways to make your ownersh...,https://www.fordpass.com/content/ford_com/fp_a...,0,0,...,0,0,0,0,0,0,2462,fordpass has so many ways to make your ownersh...,2338,407


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera         9210
Location       9226
Microphone     5447
Contacts       6398
Storage       28433
Phone          9003
SMS             295
Call_Log        194
Calendar       1049
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera        2327
Location      2296
Microphone    1339
Contacts      1536
Storage       7175
Phone         2233
SMS             57
Call_Log        33
Calendar       246
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

72684


In [23]:
corpus[0]

['obstetrics',
 'gynaecology',
 'application',
 'mainly',
 'design',
 'medical',
 'students',
 'teaching',
 'learning',
 'purposes',
 'give',
 'guide',
 'gynaecological',
 'cases']

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        obstetrics gynaecology application mainly desi...
1        fordpass many ways make ownership experience b...
2        app need internet connection read javascript c...
3        romance love couple theme note one theme lover...
4        official app sanctuary lourdes follow news go ...
                               ...                        
58142    free messages sms messages instant messaging a...
58143    best online study app competitive exams rasrps...
58144    anime aesthetic wallpapers cool app brings bes...
58145    report corruption report corruption initiative...
58146    speak app designed aid visually impaired other...
Name: Clean_Description, Length: 58147, dtype: object

In [29]:
train_sequences

[[19533,
  28339,
  8,
  3587,
  132,
  1183,
  300,
  1689,
  74,
  1392,
  153,
  503,
  52785,
  2332],
 [23323,
  49,
  1005,
  19,
  4433,
  113,
  208,
  23323,
  1,
  60,
  11010,
  712,
  10,
  17,
  120,
  672,
  65,
  528,
  799,
  120,
  2234,
  1239,
  1239,
  70,
  96,
  1681,
  3735,
  23323,
  313,
  11,
  1239,
  323,
  777,
  7455,
  444,
  868,
  218,
  2674,
  248,
  2668,
  92,
  1681,
  23323,
  313,
  11,
  23323,
  885,
  574,
  2596,
  684,
  49,
  537,
  862,
  23,
  165,
  1239,
  3219,
  9,
  1239,
  1381,
  11,
  10871,
  2499,
  799,
  192,
  19,
  262,
  690,
  7455,
  368,
  78,
  174,
  1163,
  71195,
  464,
  9209,
  77,
  973,
  2247,
  98,
  75,
  26,
  2333,
  2774,
  2930,
  2871,
  712,
  10,
  56,
  119,
  1167,
  275,
  1681,
  23323,
  313,
  1778,
  119,
  1681,
  23323,
  1,
  11010,
  751,
  192,
  291,
  712,
  10,
  77,
  23323,
  543,
  269,
  751,
  192,
  10,
  5885,
  327,
  215,
  2721,
  7981,
  631,
  3160,
  681,
  1239,
  4365,
  1

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[19533, 28339,     8, ...,     0,     0,     0],
       [23323,    49,  1005, ...,     0,     0,     0],
       [    1,    41,    89, ...,     0,     0,     0],
       ...,
       [  516,  2144,    12, ...,     0,     0,     0],
       [  760, 13430,   760, ...,     0,     0,     0],
       [  656,     1,   155, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[  45, 1394,  122, ...,    0,    0,    0],
       [ 671, 1132, 2433, ...,    0,    0,    0],
       [ 895, 2350,  609, ...,    0,    0,    0],
       ...,
       [ 195,  132,  195, ...,    0,    0,    0],
       [ 137, 1668,  369, ...,    0,    0,    0],
       [2863, 9694,  845, ...,    0,    0,    0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

obstetrics gynaecology application mainly design medical students teaching learning purposes give guide gynaecological cases
[19533, 28339, 8, 3587, 132, 1183, 300, 1689, 74, 1392, 153, 503, 52785, 2332]


In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 141112


In [36]:
word_index

{'app': 1,
 'free': 2,
 'use': 3,
 'keyboard': 4,
 'phone': 5,
 'wallpaper': 6,
 'theme': 7,
 'application': 8,
 'new': 9,
 'features': 10,
 'get': 11,
 'wallpapers': 12,
 'time': 13,
 'screen': 14,
 'also': 15,
 'best': 16,
 'like': 17,
 'one': 18,
 'make': 19,
 'download': 20,
 'android': 21,
 'easy': 22,
 'us': 23,
 'english': 24,
 'live': 25,
 'find': 26,
 'launcher': 27,
 'share': 28,
 'help': 29,
 'mobile': 30,
 'love': 31,
 'learn': 32,
 'apps': 33,
 'video': 34,
 'set': 35,
 'device': 36,
 'want': 37,
 'please': 38,
 'using': 39,
 'hd': 40,
 'need': 41,
 'support': 42,
 'game': 43,
 'home': 44,
 'enjoy': 45,
 'images': 46,
 'friends': 47,
 'simple': 48,
 'many': 49,
 'text': 50,
 'games': 51,
 'themes': 52,
 'different': 53,
 'information': 54,
 'language': 55,
 'available': 56,
 'data': 57,
 'day': 58,
 'beautiful': 59,
 'access': 60,
 'choose': 61,
 'play': 62,
 'easily': 63,
 'add': 64,
 'lock': 65,
 'create': 66,
 'way': 67,
 'you': 68,
 'every': 69,
 'status': 70,
 'words'

In [37]:
word_index["reason"]

2351

In [38]:
print(validation_sequences[0])

[45, 1394, 122, 34, 174, 451, 174, 451, 174, 4049, 9, 5412, 34, 1394, 114, 120, 1394, 50, 241, 34, 301, 512, 290, 6097, 82, 10, 710, 506, 45, 13, 451, 174, 209, 451, 174, 174, 114, 122, 47, 115, 13, 358, 365, 262, 34, 174, 237, 78, 22, 992, 527, 186, 2998, 792, 449, 3648, 2110, 824, 528, 50, 105, 241, 186, 45, 1, 101, 330, 1390, 90, 3648, 1346, 528, 527, 34, 241, 328, 123, 34, 241, 174, 90, 3155, 5086, 5086, 824, 5086, 95, 34, 241, 78, 5086, 95, 462, 53, 1772, 172, 536, 41, 2150, 54, 199, 648, 38, 142, 6765, 5396, 2581, 54, 4863, 18209, 15177, 7260, 10810, 114, 1250, 9230]


In [39]:
word_index["listen"]

419

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.30414   , -0.37029999, -0.15881   , ..., -0.37830999,
        -0.42910001,  0.0030023 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.50379997, -0.096152  , -0.13287   , ...,  0.49906999,
        -0.53661001,  0.42375001]])

In [45]:
word_index["reason"]

2351

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(58147, 600)
(58147, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(14537, 600)
(14537, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

58147
9
{0: 0.7014959584992159, 1: 0.7002794036177952, 2: 1.186116720722926, 3: 1.0098120940571707, 4: 0.22722814257298835, 5: 0.7176249892011305, 6: 21.900941619585687, 7: 33.30297823596793, 8: 6.158987395403029}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    42333900    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    42333900    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    42333900    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
1818/1818 [==============================] - 342s 182ms/step - loss: 0.1747 - accuracy: 0.2572 - val_loss: 0.2385 - val_accuracy: 0.2717
Epoch 2/10
1818/1818 [==============================] - 329s 181ms/step - loss: 0.1427 - accuracy: 0.2814 - val_loss: 0.2198 - val_accuracy: 0.2900
Epoch 3/10
1818/1818 [==============================] - 330s 182ms/step - loss: 0.1207 - accuracy: 0.2987 - val_loss: 0.2145 - val_accuracy: 0.3009
Epoch 4/10
1818/1818 [==============================] - 330s 181ms/step - loss: 0.0960 - accuracy: 0.3030 - val_loss: 0.2527 - val_accuracy: 0.3110
Epoch 5/10
1818/1818 [==============================] - 327s 180ms/step - loss: 0.0797 - accuracy: 0.3104 - val_loss: 0.2297 - val_accuracy: 0.3089
Epoch 6/10
1818/1818 [==============================] - 326s 179ms/step - loss: 0.0676 - accuracy: 0.3216 - val_loss: 0.2510 - val_accuracy: 0.2989
Epoch 7/10
1818/1818 [==============================] - 326s 179ms/step - loss: 0.0557 - accuracy: 0.3366 - val_

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2950, 1815, 26, 16, 2950, 10793, 425, 659, 2497, 1, 1, 211, 13, 16, 425, 2950, 1029, 425, 10793, 144, 425, 2950, 138, 16, 425, 2950, 1029, 425, 10793, 17, 40759, 40991, 20973, 18725, 30090, 343, 15, 211, 558, 425, 2950, 1589, 425, 2950, 425, 9754, 144, 425, 679, 425, 679, 1, 211, 16, 496, 

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[1.22639631e-05, 4.46140148e-05, 1.11439364e-08, 2.16380304e-05,
        2.43814141e-01, 7.44064106e-04, 9.61637170e-09, 1.92827126e-11,
        1.36095757e-09],
       [9.72247391e-04, 3.65617714e-04, 4.83003441e-06, 5.18707225e-07,
        1.66466013e-02, 4.35914088e-04, 9.05184123e-08, 1.01150852e-10,
        4.88774333e-07],
       [5.93493357e-02, 1.45601039e-03, 6.02541149e-01, 3.63770396e-01,
        9.86931980e-01, 3.67947072e-01, 8.29294731e-05, 8.17921682e-05,
        6.03648135e-04],
       [2.02997435e-05, 1.43186492e-03, 2.11900797e-07, 3.22261393e-01,
        6.33921266e-01, 3.15084159e-01, 1.02353533e-05, 1.35065920e-06,
        1.84448413e-06],
       [1.77569926e-01, 4.89779562e-03, 3.32946838e-05, 4.29491922e-02,
        2.94347584e-01, 6.99728310e-01, 4.82349278e-04, 2.84775288e-05,
        3.24148168e-05],
       [1.31300136e-01, 8.52553174e-03, 1.39869546e-04, 3.26071143e-01,
        9.99505401e-01, 1.20724075e-01, 4.65302901e-06, 9.07496883e-07,
        3.7

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[1.2263963e-05 4.4614015e-05 1.1143936e-08 ... 9.6163717e-09
  1.9282713e-11 1.3609576e-09]
 [9.7224739e-04 3.6561771e-04 4.8300344e-06 ... 9.0518412e-08
  1.0115085e-10 4.8877433e-07]
 [5.9349336e-02 1.4560104e-03 6.0254115e-01 ... 8.2929473e-05
  8.1792168e-05 6.0364814e-04]
 ...
 [9.7318459e-01 1.1049229e-01 6.2689399e-03 ... 3.3206117e-05
  1.6700242e-05 6.8062152e-05]
 [5.7644123e-05 9.9991405e-01 9.9861383e-01 ... 6.8346565e-12
  3.3458453e-12 1.6918930e-05]
 [4.3130917e-06 1.0635833e-05 4.3698611e-07 ... 2.2637955e-06
  1.9633746e-09 3.8638623e-07]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.198 with F-Score: 0.659
permission  1
Length of sequence: 10000
Best Threshold: 0.1252 with F-Score: 0.5679
permission  2
Length of sequence: 10000
Best Threshold: 0.0287 with F-Score: 0.5695
permission  3
Length of sequence: 10000
Best Threshold: 0.616 with F-Score: 0.5192
permission  4
Length of sequence: 10000
Best Threshold: 0.4492 with F-Score: 0.7984
permission  5
Length of sequence: 10000
Best Threshold: 0.4472 with F-Score: 0.5318
permission  6
Length of sequence: 10000
Best Threshold: 0.0545 with F-Score: 0.2
permission  7
Length of sequence: 10000
Best Threshold: 0.4186 with F-Score: 0.5
permission  8
Length of sequence: 10000
Best Threshold: 0.0517 with F-Score: 0.3758
-------------------------------------
optimal threshold tuning for f-score
[0.198, 0.1252, 0.0287, 0.616, 0.4492, 0.4472, 0.0545, 0.4186, 0.0517]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.061500001698732376 with G-Mean: 0.8387
FPR: 0.1366, TPR: 0.8148
permission  1
Best Threshold: 0.01549999974668026 with G-Mean: 0.7644
FPR: 0.2106, TPR: 0.7402
permission  2
Best Threshold: 0.0005000000237487257 with G-Mean: 0.8223
FPR: 0.1765, TPR: 0.8211
permission  3
Best Threshold: 0.026399999856948853 with G-Mean: 0.7863
FPR: 0.2515, TPR: 0.826
permission  4
Best Threshold: 0.8665000200271606 with G-Mean: 0.7828
FPR: 0.1854, TPR: 0.7523
permission  5
Best Threshold: 0.04809999838471413 with G-Mean: 0.7673
FPR: 0.2689, TPR: 0.8052
permission  6
Best Threshold: 0.0007999999797903001 with G-Mean: 0.8605
FPR: 0.0949, TPR: 0.8182
permission  7
Best Threshold: 0.0 with G-Mean: 0.9156
FPR: 0.1618, TPR: 1.0
permission  8
Best Threshold: 0.00019999999494757503 with G-Mean: 0.8241
FPR: 0.185, TPR: 0.8333
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0615, 0.0155, 0.0005, 0.0264, 0.8665, 0.0481, 0.0008, 0.0, 0.0002]


## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6590
  Location  : 0.5679
  Microphone: 0.5695
  Contacts  : 0.5192
  Storage   : 0.7984
  Phone     : 0.5318
  SMS       : 0.2000
  Call_Log  : 0.5000
  Calendar  : 0.3758

  Average F1 score: 0.5246


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8774
  Location  : 0.8549
  Microphone: 0.9163
  Contacts  : 0.8919
  Storage   : 0.7753
  Phone     : 0.8629
  SMS       : 0.9931
  Call_Log  : 0.9991
  Calendar  : 0.9799

  Average F1 (micro) score: 0.9056


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8385
  Location  : 0.7647
  Microphone: 0.8211
  Contacts  : 0.7863
  Storage   : 0.7834
  Phone     : 0.7679
  SMS       : 0.8614
  Call_Log  : 0.5000
  Calendar  : 0.8191

  Average ROC_AUC score: 0.7714


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.674 with F-Score: 0.4541
permission  1
Length of sequence: 10000
Best Threshold: 0.1324 with F-Score: 0.5778
permission  2
Length of sequence: 10000
Best Threshold: 0.2126 with F-Score: 0.5208
permission  3
Length of sequence: 10000
Best Threshold: 0.1698 with F-Score: 0.6064
permission  4
Length of sequence: 10000
Best Threshold: 0.0232 with F-Score: 0.5933
permission  5
Length of sequence: 10000
Best Threshold: 0.9135 with F-Score: 0.431
permission  6
Length of sequence: 10000
Best Threshold: 0.0017 with F-Score: 0.5635
permission  7
Length of sequence: 10000
Best Threshold: 0.0057 with F-Score: 0.3833
permission  8
Length of sequence: 10000
Best Threshold: 0.0117 with F-Score: 0.4784
-------------------------------------
optimal threshold tuning for f-score
[0.674, 0.1324, 0.2126, 0.1698, 0.0232, 0.9135, 0.0017, 0.0057, 0.0117]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.08619999885559082 with G-Mean: 0.7066
FPR: 0.2717, TPR: 0.6856
permission  1
Best Threshold: 0.05950000137090683 with G-Mean: 0.7475
FPR: 0.2279, TPR: 0.7237
permission  2
Best Threshold: 0.00279999990016222 with G-Mean: 0.77
FPR: 0.2235, TPR: 0.7635
permission  3
Best Threshold: 0.17470000684261322 with G-Mean: 0.7144
FPR: 0.2952, TPR: 0.7241
permission  4
Best Threshold: 0.9199000000953674 with G-Mean: 0.589
FPR: 0.3763, TPR: 0.5563
permission  5
Best Threshold: 0.40369999408721924 with G-Mean: 0.7795
FPR: 0.2631, TPR: 0.8246
permission  6
Best Threshold: 0.0005000000237487257 with G-Mean: 0.8045
FPR: 0.2205, TPR: 0.8302
permission  7
Best Threshold: 9.999999747378752e-05 with G-Mean: 0.805
FPR: 0.2595, TPR: 0.875
permission  8
Best Threshold: 0.00139999995008111 with G-Mean: 0.8656
FPR: 0.171, TPR: 0.9038
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0862, 0.0595, 0.0028, 0.1747, 0.9199, 0.4037, 0.0005, 1e-04, 0.0014]

#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4541
  Location  : 0.5778
  Microphone: 0.5208
  Contacts  : 0.6064
  Storage   : 0.5933
  Phone     : 0.4310
  SMS       : 0.5635
  Call_Log  : 0.3833
  Calendar  : 0.4784

  Average F1 score: 0.5121


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8317
  Location  : 0.7984
  Microphone: 0.9024
  Contacts  : 0.7107
  Storage   : 0.4745
  Phone     : 0.9066
  SMS       : 0.8444
  Call_Log  : 0.8953
  Calendar  : 0.9059

  Average F1 (micro) score: 0.8078


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV5_N72k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7047
  Location  : 0.7462
  Microphone: 0.7674
  Contacts  : 0.7145
  Storage   : 0.5900
  Phone     : 0.7764
  SMS       : 0.7947
  Call_Log  : 0.7989
  Calendar  : 0.8660

  Average ROC_AUC score: 0.7510
